## 推荐系统

在当前互联网生活中我们经常会碰到推荐问题，比如穿衣推荐、美食推荐、新闻推荐等等，如何根据用户信息、商品信息、用户行为信息给一个用户推荐他可能感兴趣的商品，对于促进销售具有非常重要的意义。

目前流行的推荐算法当属协同过滤算法，该算法利用了用户和商品信息，如下表所示，假设共有7个用户和5部电影，用户对电影的评分取值范围为[1,5]，0表示用户尚未评分。假如给定用户3，我们应该推荐哪部电影给他呢？



| | Movie 1| Movie 2 | Movie 3 | Movie 4 | Movie 5|
|--|--|--|--|--|--|
| User 1| 4 | 4 | 0 | 2 | 2 |
| User 2| 4 | 0 | 0 | 3 | 3 |
| User 3| 4 | 0 | 0 | 1 | 1 |
| User 4| 1 | 1 | 1 | 2 | 0 |
| User 5| 2 | 2 | 2 | 0 | 0 |
| User 6| 1 | 1 | 1 | 0 | 0 |
| User 7| 5 | 5 | 5 | 0 | 0 |



协同过滤算法的关键在于相似度算法。上述“用户-电影”矩阵，我们可以把每一行看作是一个用户向量，每一列看作是电影向量，当我们计算用户向量之间的相似度时，称为**基于用户的协同过滤**；当我们计算电影向量之间的相似度时，称为**基于物品的协同过滤**。通常用户数量远远大于商品数量，所以我们一般选择基于物品的协同过滤，算法具体计算过程如下所示，首先我们计算用户3对电影2的预测评分，分别计算电影2和电影1、电影4和电影5的相似度得到$s_1, s_4, s_5$，之后通过一个加权平均得到用户3对电影2的预测评分为2.02；同理得到对电影3的预测评分为2.5。

\begin{equation}
\begin{aligned}
s_1 = & CosSim(\begin{bmatrix}
4 \\ 
1 \\ 
2 \\ 
1 \\
5
\end{bmatrix},\begin{bmatrix}
4 \\ 
1 \\ 
2 \\ 
1 \\
5
\end{bmatrix})=1 \\
s_4 = & CosSim(\begin{bmatrix}
2 \\ 
2
\end{bmatrix},\begin{bmatrix}
4 \\ 
1
\end{bmatrix})=0.93 \\
s_5 = & CosSim(2,4)=1
\end{aligned}
\end{equation}


得到Movie 2的评分为：
$$
Score(\text{Movie 2})=\frac{4*1+1*0.93+1*1}{1+0.93+1}=2.02
$$

同理可计算Moview 3的评分为：
$$
Score(\text{Movie 3})=2.50
$$

所以给用户3推荐的电影顺序为：Movie 3、Movie 2。


## SVD和推荐

我们知道，通过SVD可以将一个矩阵分解为3个矩阵：
$$
M_{m\times n} \approx U_{m\times k}\Sigma_{k\times k} V^T_{k \times n}
$$

然后对于“用户-电影”矩阵$M$中的每一个评分$m_{ij}$可以用$u_i\Sigma v_j$表示。那么问题来了，SVD只是对矩阵进行了分解，哪里起到预测作用呢？矩阵中是0的地方，经过$u_i\Sigma v_j$也还是0。

确实，通常我们说利用SVD进行推荐，并不是说直接对矩阵$M$进行分解进行预测，而是使用FunkSVD方法，这个算法借鉴了矩阵分解的思想。FunkSVD是Simon Funk在Netflix电影推荐大赛中使用的方法，下面详细介绍此算法思路：



既然SVD可以将一个矩阵分解为3个矩阵相乘，那是否有方法可以将一个矩阵分解为两个矩阵相乘？如：

$$
M_{m\times n}=P_{m\times k}Q_{k \times n}
$$

这样对于每一个用户评分$m_{ij}$可以用$p_{i}q_{j}$表示。那怎么得到P和Q呢？这里运用了规划的思想，因为我们的目标是使得$m_{ij}$和$p_{i}q_{j}$之间差距尽可能小，所以损失函数定义为均方差：

$$
J(p_i,q_j)=\frac{1}{2mn}\sum_{i,j}(p_{i}q_{j}-m_{ij})^2
$$

这样就可以使用传统的最优化算法得到$P$和$Q$了。注意这里$p_{i}$和$q_{j}$均为向量，训练获得参数个数为$mk+kn$。